Dependencies | Version
--- | ---
SatPy | 0.9.3
PyResample | 1.10.1
PySpectral | 0.8.2
Trollimage | 1.5.7
PyKdtree | 1.3.1
XArray | 0.10
Dask | 0.18.2
ImageIO | 2.3.0
ffmpeg | 3.4


# GOES-16 ABI - True Color Animation - Hurricane Florence



This example shows how to take multiple time steps of ABI data and combine them in to a single mp4 movie file as an animation. The example uses 100 frames of mesoscale images from the GOES-16 ABI instrument on September 11th, 2018 from 1300Z to 1440Z with one image every minute. The "%%time" magic command is used in jupyter notebook to time the entire cell's execution. This example took about 8 minutes and 30 seconds on a 2015 macbook using 8 dask threaded workers.

The movie produced has been hosted on twitter: https://twitter.com/PyTrollOrg/status/1039555399433834497

Full resolution video: https://www.ssec.wisc.edu/~davidh/satpy/true_color_20180911_130021.mp4

The frames in the movie are 500m resolution true color images with solar zenith and rayleigh scattering corrections applied. A ratio sharpening technique was also used to bring out more detail from the lower resolution input bands. All of this is pre-configured for the ABI 'true_color' composite in satpy and is done from python.

In [1]:
%%time
import os
from satpy import Scene
from glob import glob
from satpy.multiscene import MultiScene

BASE_DIR = '/data/data/abi/20180911_florence'
all_filenames = [glob(fn.replace('C01', 'C0[123]*')[:len(BASE_DIR) + 50] + '*.nc') for fn in sorted(glob(os.path.join(BASE_DIR, 'OR*-RadM1-*C01*.nc')))]
scenes = [Scene(reader='abi_l1b', filenames=filenames) for filenames in all_filenames]
print("Number of Scenes: ", len(scenes))

mscn = MultiScene(scenes)
mscn.load(['true_color'])
new_mscn = mscn.resample(resampler='native')
new_mscn.save_animation('/tmp/{name}_{start_time:%Y%m%d_%H%M%S}.mp4', fps=5)

Number of Scenes:  100


Platform file /Users/davidh/repos/git/satpy/satpy/etc/platforms.txt not found.
/Users/davidh/anaconda/envs/polar2grid_py36/lib/python3.6/site-packages/dask/local.py:255: RuntimeWarning: invalid value encountered in log10
  return func(*args2)
/Users/davidh/anaconda/envs/polar2grid_py36/lib/python3.6/site-packages/dask/local.py:255: RuntimeWarning: invalid value encountered in log10
  return func(*args2)


CPU times: user 13min 53s, sys: 6min 2s, total: 19min 55s
Wall time: 8min 29s
